<hr style="border:0.2px solid black"> </hr>

<figure>
  <IMG SRC="img/ntnu-norwegian-university-of-science-and-technology-vector-logo.png" WIDTH=250 ALIGN="right">
</figure>

**<ins>Course:</ins>** TVM4174 - Hydroinformatics for Smart Water Systems

# <ins>Group project:</ins> Working with SWMM in Python - Part 2: Running and studying results
   
    
*Developed by Lucia Arce and Magdalena Jaurena*

<hr style="border:0.2px solid black"> </hr>


In [ ]:
## PySWMM installation - package used to run the model ##
!pip install pyswmm

from pyswmm import Simulation

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import datetime
sns.set_style('whitegrid')

## 1- Run the model with PySWMM

In [ ]:
m_mod = Simulation('SWMM_model.inp') # Change the model in case it is modified.
m_mod.execute()

"When a run completes successfully, the mass continuity errors for runoff, flow routing, and pollutant routing will be displayed in the Run Status window. These errors represent the percent difference between initial storage + total inflow and final storage + total outflow for the entire drainage system. If they exceed some reasonable level, such as 10 percent, then the validity of the analysis results must be questioned. The most common reasons for an excessive continuity error are computational time steps that are too long or conduits that are too short." Source: EPA SWMM

In [ ]:
print(f"\n The flow routing mass balance error is {m_mod.flow_routing_error} ")
print(f"\n The pollutant routing mass error is {m_mod.quality_error} ")
print(f"\n The runouff routing mass error is {m_mod.runoff_error} ")

## 2-  Study the results with SWMMIO

In [ ]:
!pip install swmmio
import swmmio

In [ ]:
m_mod = swmmio.Model('SWMM_model.inp')


In [ ]:
### Needed information of the network ###

## Nodes ##

nodes=m_mod.inp.junctions
nodes_coord=m_mod.inp.coordinates

## Links ##

links=m_mod.links.dataframe
conduits=m_mod.inp.conduits

##  outfalls ##
outfalls=m_mod.inp.outfalls

## storages ##
storages=m_mod.inp.storage

## weirs ##

weirs=m_mod.inp.weirs

In [ ]:
### Links results ###

links=m_mod.links.dataframe
print(f'The information stored for the links is: {links.columns}')
links_df=(links[['MaxQ','MaxV']])

In [ ]:
### Code extracted from course "Urban Water Systems NTNU - 2021"  and adapted ###
### PLOT SLOPES ###

polygons = m_mod.inp.polygons
catchment_list = np.unique(polygons.index.to_numpy())  #Obtain catchment´s list
manhole_list = np.unique(nodes_coord.index.to_numpy()) #Obtain nodes´s list

fig, ax = plt.subplots(figsize = (10,10)) 

for catchment in catchment_list: 
    ax.fill(polygons.loc[catchment,"X"],polygons.loc[catchment,"Y"],ec = "burlywood", fc="floralwhite")
    
## Plot Conduits Velocity ##
for link in links_df.index: 
    inlet = nodes_coord.loc[links.loc[link, "InletNode"]] 
    outlet = nodes_coord.loc[links.loc[link, "OutletNode"]] 
    link_x = [inlet.X, outlet.X] 
    link_y = [inlet.Y, outlet.Y]
    if links_df.loc[link,'MaxV']>=5:
        ax.plot(link_x, link_y, c='r', linewidth=3)
        link_x_5=link_x
        link_y_5=link_y
    else:
        ax.plot(link_x, link_y, c='gray', linewidth=3)
        link_x_=link_x
        link_y_=link_y
        
## Plot Weirs ##
for i in weirs.index: 
    inlet = nodes_coord.loc[weirs.loc[i, "InletNode"]] 
    outlet = nodes_coord.loc[weirs.loc[i, "OutletNode"]] 
    x_W = [inlet.X, outlet.X] 
    y_W = [inlet.Y, outlet.Y] 
    ax.plot(x_W,y_W, color = "b", linestyle='-.')

## Plot Nodes ##
for i in nodes.index: 
    coord_n = nodes_coord.loc[i] 
    ax.scatter(coord_n['X'], coord_n['Y'], color = "k", zorder=3 )
    
## Plot Outfalls ##
for i in outfalls.index: 
    coord_Out = nodes_coord.loc[i] 
    ax.scatter(coord_Out['X'],coord_Out['Y'], color = "k", marker='^', zorder=3)

## Plot Storages ##
for i in storages.index: 
    coord_S = nodes_coord.loc[i] 
    ax.scatter(coord_S['X'], coord_S['Y'], color = "red", marker='s', s=200, zorder=3)
    
    
plt.title('Link velocity', fontsize=22)
plt.xlabel('x coordinate', fontsize=16)
plt.ylabel('y coorinate', fontsize=16)

# Create legend #
ax.plot(link_x_5,link_y_5, color = "r", linewidth=2 , label='Velocity >= 5 m/s')
ax.plot(link_x_,link_y_, color = "gray", linewidth=2 , label='Slope < 5 m/s')
# ax.plot(slope_x_05,slope_y_05, color = "b", linewidth=2 , label='Slope < 0.5%')
# ax.plot(slope_x, slope_y, c='k', linewidth=2, label='No calculated slope')
ax.plot(x_W,y_W, color = "k", linestyle='-.', label='Weirs')
ax.scatter(coord_n['X'], coord_n['Y'], color = "k", label='Nodes', zorder=3 )
ax.scatter(coord_Out['X'],coord_Out['Y'], color = "k", marker='^', zorder=3, label='Outfalls')
ax.scatter(coord_S['X'], coord_S['Y'], color = "red", marker='s', s=200, label='Storages', zorder=3)


plt.legend(loc='best', frameon=False, title='Elements', title_fontsize=14)
plt.show()

In [ ]:
### FLOODING ###

Flooding = m_mod.rpt.node_flooding_summary
print (f'The information stored related to FLOODING is: {Flooding.columns}')

flood=(Flooding[['MaxQFlooding','TotalFloodVol']])

(i,ii)=flood.sum(axis=0)

print (f'\nThe total flooding volume is: {ii} m3')

# NODES WITH MAXIMUM FLOOD FLOW OR VOLUME #

Q_max_node=flood.idxmax()['MaxQFlooding']
V_max_node = flood.idxmax()['TotalFloodVol']

print (f'\nThe node where the maximum flood peak is happening is: {Q_max_node}')
print (f'\nThe node with maximum flood volumen is: {V_max_node}')

print (f'\n{len(flood)} nodes from {len(nodes)} have flooding problem')

In [ ]:
### PLOT FLOODED NODES ###

### Code extracted from course "Urban Water Systems NTNU - 2021"  and adapted ###

polygons = m_mod.inp.polygons
catchment_list = np.unique(polygons.index.to_numpy())  #Obtain catchment´s list
manhole_list = np.unique(nodes_coord.index.to_numpy()) #Obtain nodes´s list

fig, ax = plt.subplots(figsize = (10,10)) 

for catchment in catchment_list: 
    ax.fill(polygons.loc[catchment,"X"],polygons.loc[catchment,"Y"], ec = "burlywood", fc="floralwhite")
    
## Plot Conduits ##
for link in conduits.index: 
    inlet = nodes_coord.loc[conduits.loc[link, "InletNode"]] 
    outlet = nodes_coord.loc[conduits.loc[link, "OutletNode"]] 
    x_l = [inlet.X, outlet.X] 
    y_l = [inlet.Y, outlet.Y] 
    ax.plot(x_l,y_l, color = "k")

## Plot Weirs ##
for i in weirs.index: 
    inlet = nodes_coord.loc[weirs.loc[i, "InletNode"]] 
    outlet = nodes_coord.loc[weirs.loc[i, "OutletNode"]] 
    x_W = [inlet.X, outlet.X] 
    y_W = [inlet.Y, outlet.Y] 
    ax.plot(x_W,y_W, color = "b", linestyle='-.')

## Plot Flooded nodes ##
for i in flood.index: 
    coord_n = nodes_coord.loc[i] 
    size=flood.loc[i, "MaxQFlooding"]
    ax.scatter(coord_n['X'], coord_n['Y'], s=size*2 ,color = "aqua", zorder=3)
    
## Plot Nodes ##
for i in nodes.index: 
    coord_n = nodes_coord.loc[i] 
    ax.scatter(coord_n['X'], coord_n['Y'], s=10 , color = "k", zorder=4 )
    
## Plot Outfalls ##
for i in outfalls.index: 
    coord_Out = nodes_coord.loc[i] 
    ax.scatter(coord_Out['X'],coord_Out['Y'], color = "k", marker='^', zorder=3)

## Plot Storages ##
for i in storages.index: 
    coord_S = nodes_coord.loc[i] 
    ax.scatter(coord_S['X'], coord_S['Y'], color = "red", marker='s', s=200, zorder=3)
    
    
plt.title('Flooded nodes', fontsize=22)
plt.xlabel('x coordinate', fontsize=16)
plt.ylabel('y coorinate', fontsize=16)

# Create legend #
ax.plot(x_l,y_l, color = "k", label='Conduits')
ax.plot(x_W,y_W, color = "k", linestyle='-.', label='Weirs')
ax.scatter(coord_n['X'], coord_n['Y'], color = "aqua", label='Flooded nodes', zorder=3 )
ax.scatter(coord_n['X'], coord_n['Y'], color = "k", label='Nodes', zorder=3 )
ax.scatter(coord_Out['X'],coord_Out['Y'], color = "k", marker='^', zorder=3, label='Outfalls')
ax.scatter(coord_S['X'], coord_S['Y'], color = "red", marker='s', s=200, label='Storages', zorder=3)


plt.legend(loc='best', frameon=False, title='Elements', title_fontsize=14)
plt.show()

In [ ]:
### Subcatchments results ###

subcatchments = m_mod.rpt.subcatchment_runoff_summary #from here we can get the total precipitation and total infiltration
print (f'The information stored related to SUBCATCHMENTS is: {subcatchments.columns}')


In [ ]:
subcatch_df=(subcatchments[['TotalPrecip','TotalRunoffIn','TotalInfil','TotalEvap']])
(total_precip,total_run, total_inf,Total_evap) = subcatch_df.sum(axis=0)


print(f"\n The total precipitation is {total_precip:.2f} ")
print(f"\n The total runoff is {total_run:.2f} ")
print(f"\n The total infiltration is {total_inf:.2f} ")
print(f"\n The total evaporation is {Total_evap:.2f} ")



# Conclusions:

### - Difficulties to work with PySWMM, not intuitive.
### - Difficulties to run the model in SWMMIO.
### - Packages not able to read the .out file given by SWMM.
### - Poor information in the .rpt file (not hydrograms inlet/outlet). Just summary and maximum values.

